In [22]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import pingouin as pg
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf

df = pd.read_csv("1st-dataset-prepressed-baseball.csv")     # 1차 데이터셋

pd.set_option("display.max_columns", None)

# 분석용 편의 함수
def pct(n): return f"{n:.1%}"

# df['accident_score'] = df['accident_count'] + 3 * df['injury_count'] + 10 * df['death_count']
df['accident_score'] = df['accident_count'] + 3 * df['injury_count']


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072 entries, 0 to 2071
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   game_id         2072 non-null   int64  
 1   stadium_code    2072 non-null   object 
 2   sports_type     2072 non-null   object 
 3   game_date       2072 non-null   object 
 4   day_of_week     2072 non-null   object 
 5   is_holiday      2072 non-null   int64  
 6   start_time      2072 non-null   object 
 7   home_team_win   2072 non-null   int64  
 8   audience        2072 non-null   float64
 9   region          2072 non-null   object 
 10  accident_count  2072 non-null   float64
 11  injury_count    2072 non-null   float64
 12  death_count     2072 non-null   float64
 13  temperature     2072 non-null   float64
 14  precipitation   2072 non-null   float64
 15  snow_depth      2072 non-null   float64
 16  start_hour      2072 non-null   int64  
 17  match_시범경기      2072 non-null   b

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# X-Columes
x_cols = ['match_정규시즌', 'match_포스트시즌', 'is_holiday', 'start_hour', 'audience'] 

# 1. 독립 변수와 종속 변수 정의
X = df.drop(columns=["accident_score", "accident_count", "injury_count", "death_count", "game_id", "stadium_code","sports_type","game_date", "day_of_week", "start_time", "region", "snow_depth"])
y = df['accident_score']

# 2. 학습용/테스트용 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [24]:
import statsmodels.api as sm
import pandas as pd

# X: 특성(피처), y: 타겟값 (예: 교통사고 건수)
# X는 반드시 상수항 추가 필요 (절편 포함)
# X = df[['match_정규시즌', 'match_포스트시즌', 'is_holiday', 'start_hour', 'audience']]
# y = df['accident_count']  # 예측 대상

# bool 값을 int 형으로!!!
X = X.astype({col: int for col in X.select_dtypes(include='bool').columns})

X = sm.add_constant(X)  # 절편항 추가

# 모델 적합
model = sm.OLS(y, X).fit()

# 분석표 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         accident_score   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     8.267
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           4.42e-11
Time:                        14:29:54   Log-Likelihood:                -9454.0
No. Observations:                2072   AIC:                         1.893e+04
Df Residuals:                    2063   BIC:                         1.898e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.3822      6.138      0.714



# 야구경기일 사고위험도 (`accident_score`) 예측 회귀모델 보고서

## 1. 모델 개요

* **목적**: 야구 경기 관련 요인이 교통사고 위험도(`accident_score`)에 미치는 영향을 정량적으로 분석
* **모델**: OLS (Ordinary Least Squares) 선형 회귀 분석
* **종속변수**: `accident_score` -> accident_count + injury_count*3 (사고 건수 및 부상자수를 조합.) 
* **독립변수**:

  * 경기 정보: `home_team_win`, `match_시범경기`, `match_정규시즌`, `match_포스트시즌`, 'audience'
  * 시간·환경 변수: `start_hour`, `temperature`, `precipitation`
  * 기타 요인: `is_holiday`

---

## 2. 모델 적합도

| 항목             | 값             | 해석                               |
| -------------- | ------------- | -------------------------------- |
| R-squared      | **0.031**     | 모델이 전체 사고위험도 변동의 약 **3.1%** 를 설명 |
| Adj. R-squared | 0.027         | 변수 수 고려한 설명력은 2.7%로 매우 낮음        |
| F-statistic    | 8.267         | 모델 전체는 통계적으로 **유의함** (p < 0.001) |
| AIC / BIC      | 18930 / 18980 | 모델 적합도 비교 지표로 활용 가능              |

➡️ **해석**: 모델은 유의하지만 설명력은 매우 낮아, 주요 설명 변수 누락 가능성 높음

---

## 3. 계수 해석 (Coefficients)

| 변수명              | 계수     | p-value   | 유의성  | 해석                                 |
| ---------------- | ------ | --------- | ---- | ---------------------------------- |
| **is\_holiday**  | -3.17  | **0.028** | O | 공휴일일 경우 평균적으로 사고위험도가 **3.17점 낮음**  |
| **audience**     | 0.0006 | **0.000** | O | 관중 수가 증가할수록 사고위험도 **비례 증가**        |
| **start\_hour**  | 1.02   | **0.037** | O | 경기 시작이 1시간 늦을수록 사고위험도 **1.02점 증가** |
| **match\_포스트시즌** | 8.99   | **0.018** | O | 포스트시즌 경기는 사고위험도가 **약 9점 증가**       |
| home\_team\_win  | 0.55   | 0.589     | X | 승패 여부는 사고위험도에 영향 없음                |
| temperature      | -0.11  | 0.259     | X    | 기온은 유의하지 않음                        |
| precipitation    | -0.51  | 0.115     | X    | 강수량도 유의하지 않음                       |
| match\_시범경기      | -3.13  | 0.127     | X    | 시범경기는 위험도 낮지만 통계적으로 유의하지 않음        |
| match\_정규시즌      | -1.49  | 0.617     | X    | 정규시즌 여부는 유의하지 않음                   |

➡ **핵심 변수**: `is_holiday`, `audience`, `start_hour`, `match_포스트시즌`

---

## 4. 잔차 분석 및 모델 한계

* R² 값이 0.03으로 **설명력이 매우 낮음**
* 비록 일부 변수는 유의하지만, **전체적 예측력은 낮아 실무 적용에는 한계**
* 모델 잔차 시각화 및 이상치 탐지, 변수 변환 또는 비선형 모델 사용 고려 필요

---

## 5. 결론 및 제언

### 결론

* 공휴일, 관중 수, 경기 시작 시간, 포스트시즌 여부가 **사고위험도에 유의미한 영향**
* 일반적인 정규시즌 여부, 기온, 강수량, 시범경기 여부 등은 영향 미미
* 높은 사고위험도는 **포스트시즌**, **늦은 경기 시작 시간**, **많은 관중**과 관련 있음

### 제언

1. **설명력 개선**:

   * 교통량, 경기장 위치, 주차시설, 지역별 통제 변수 추가
   * 시간대 별 혼잡도나 주변 시설 이벤트 고려

2. **모델 대안 제시**:

   * 비선형 모델 (e.g. Random Forest, Gradient Boosting)
   * 이산형 사고건수를 직접 예측하는 포아송 회귀 또는 음이항 회귀(Negative Binomial)

3. **지표 정교화**:

   * `accident_score` 산정 방식이 다중 요인을 반영하는 만큼 변수간 상호작용도 고려 필요


